##### https://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work

In [6]:
https://github.com/marlhakizi/Absenteeism-time-in-hours/blob/master/Absentee.ipynb
    
import pandas as pd
import numpy as np


from sklearn.svm import SVC
import matplotlib.pyplot as plt


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection  import cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [3]:
df = pd.read_excel('Part 3 -  Absenteeism_at_work.xls')
print(df.shape)


(741, 21)


In [4]:
# print(df.isna().sum())
# print(df.nunique())
# print(df.head(),df.tail())

In [5]:
#drop last row = all null
df.drop(740,inplace=True)
df.describe().T
df.shape

(740, 21)

In [ ]:
# X = df.drop('Absenteeism time in hours',axis=1)
# y = df['Absenteeism time in hours']

# X_train,X_test,y_train ,y_test = train_test_split(X,y,random_state = 42,train_size = 0.8,test_size=0.2)
# #X_train,X_test ,y_train ,y_test

# lin_model = LinearRegression()
# lin_model.fit(X_train,y_train)


# # Make predictions using the testing set
# y_pred = lin_model.predict(X_test)
# y_pred_train = lin_model.predict(X_train)

# print('train root Mean squared error: %.2f'
#       % np.sqrt(mean_squared_error(y_train, y_pred_train)))

# print('test Mean squared error: %.2f'
#       % np.sqrt(mean_squared_error(y_test, y_pred)))
# lin_model.score(X_train,y_train)

In [11]:
df_duplicated = df.copy()
display(df.shape)
for idx, row in df.iterrows():
    hours = row.loc['Absenteeism time in hours']
    if hours > 8:
        days = int(hours/8)
        row_tmp = row
        row_tmp.loc['Absenteeism time in hours'] = 8
        day_of_week = row.loc['Day of the week'] 
        for i in range(days):
            row_tmp.loc['Day of the week']  = day_of_week 
            df_duplicated = df_duplicated.append([row_tmp],ignore_index=True)
            if (day_of_week == 6):
                day_of_week = 2
            else:
                day_of_week += 1
        df_duplicated.drop(idx,inplace=True)

print(df_duplicated.shape)
display(df_duplicated)

(740, 21)

(982, 21)


In [29]:
#df_duplicated[df_duplicated['Disciplinary failure']==1] -> all Absenteeism time in hours == 0

#A closer look of the Absenteeism time in hours and Reason for absence columns reveals that all 0's in both 
#Absenteeism time in hours,Month of absence and Reason for absence columns correspond to Absenteeism time
#in hours=0.Doing so also takes out all the Disciplinary failure==1 leaving us with only Disciplinary failure==0.
#This means that this feature is no longer needed for the analysis.

df_2=df_duplicated[df_duplicated['Absenteeism time in hours']!=0]
df_2=df_duplicated.drop(['Disciplinary failure'],axis=1)
len(df_2.ID.unique())

35

In [37]:
df_2.describe()


,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
count,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000,982.000000
mean,18.021385,17.443992,6.404277,3.960285,2.554990,219.766802,27.937882,12.964358,37.114053,272457.953157,94.636456,1.249491,1.128310,0.607943,0.063136,0.643585,79.257637,173.238289,26.393075,7.385947
std,10.597470,8.027883,3.383491,1.410407,1.105325,68.726226,14.628851,7.575630,7.245458,40483.458288,3.919061,0.627175,1.111442,0.488458,0.243332,1.163168,13.097681,7.385983,4.084764,11.457973
min,1.000000,0.000000,0.000000,2.000000,1.000000,118.000000,5.000000,-9.000000,27.000000,205917.000000,81.000000,1.000000,0.000000,0.000000,0.000000,0.000000,56.000000,163.000000,19.000000,0.000000
25%,10.000000,12.000000,3.000000,3.000000,2.000000,179.000000,14.000000,10.000000,32.000000,241476.000000,93.000000,1.000000,0.000000,0.000000,0.000000,0.000000,68.000000,170.000000,23.000000,2.000000
50%,17.000000,19.000000,6.000000,4.000000,3.000000,225.000000,26.000000,13.000000,37.000000,264604.000000,95.000000,1.000000,1.000000,1.000000,0.000000,0.000000,83.000000,171.000000,25.000000,8.000000
75%,28.000000,23.000000,9.000000,5.000000,4.000000,260.000000,36.000000,16.000000,40.000000,294217.000000,98.000000,1.000000,2.000000,1.000000,0.000000,1.000000,89.000000,172.000000,30.000000,8.000000
max,36.000000,28.000000,12.000000,6.000000,4.000000,388.000000,52.000000,111.000000,58.000000,378884.000000,100.000000,4.000000,4.000000,1.000000,1.000000,8.000000,108.000000,196.000000,38.000000,120.000000


In [32]:
transport=df_2.groupby(['ID'])['Transportation expense'].mean()
distance=df_2.groupby(['ID'])['Distance from Residence to Work'].mean()
service_time = df_2.groupby(['ID'])['Service time'].mean()
age = df_2.groupby(['ID'])['Age'].mean()
work_load_avg = df_2.groupby(['ID'])['Work load Average/day '].mean()
hit_target = df_2.groupby(['ID'])['Hit target'].mean()
education=df_2.groupby(['ID'])['Education'].mean()
son=df_2.groupby(['ID'])['Son'].mean()
drink=df_2.groupby(['ID'])['Social drinker'].mean()
smoker=df_2.groupby(['ID'])['Social smoker'].mean()
pet=df_2.groupby(['ID'])['Pet'].mean()
weight=df_2.groupby(['ID'])['Weight'].mean()
height=df_2.groupby(['ID'])['Height'].mean()
body=df_2.groupby(['ID'])['Body mass index'].mean()

hours=df_2.groupby(['ID'])['Absenteeism time in hours'].sum()

df_3 = pd.concat([transport,distance,service_time,age,work_load_avg,hit_target,education,son,drink,smoker,pet,weight,height,
                 body,hours],axis=1)

df_3

ID
1.0     95.136364
2.0     92.800000
3.0     95.217742
4.0     95.000000
5.0     92.333333
6.0     94.500000
7.0     94.714286
8.0     95.000000
9.0     95.891892
10.0    93.555556
11.0    94.289855
12.0    96.000000
13.0    92.777778
14.0    96.626667
15.0    91.584906
16.0    97.500000
17.0    95.869565
18.0    95.350000
19.0    89.333333
20.0    94.745455
21.0    94.000000
22.0    93.966667
23.0    96.714286
24.0    92.659574
25.0    95.600000
26.0    97.909091
27.0    95.142857
28.0    94.378049
29.0    98.333333
30.0    93.714286
31.0    96.666667
32.0    92.600000
33.0    94.285714
34.0    95.400000
36.0    95.229508
Name: Hit target, dtype: float64


,Hit target,Work load Average/day,Age,Transportation expense,Service time,Height,Body mass index,Weight,Social drinker,Social smoker,Education,Pet,Distance from Residence to Work,Absenteeism time in hours
ID,,,,,,,,,,,,,,
1.0,95.136364,275948.181818,37.0,235.0,14.000000,172.0,29.0,88.0,0.0,0.0,3.0,1.0,11.0,122.0
2.0,92.800000,224626.000000,48.0,235.0,12.000000,163.0,33.0,88.0,0.0,1.0,1.0,5.0,29.0,25.0
3.0,95.217742,264725.330645,38.0,179.0,19.266129,170.0,31.0,89.0,1.0,0.0,1.0,0.0,51.0,594.0
4.0,95.000000,271219.000000,40.0,118.0,13.000000,170.0,34.0,98.0,1.0,0.0,1.0,8.0,14.0,0.0
5.0,92.333333,265689.000000,43.0,235.0,13.000000,167.0,38.0,106.0,1.0,0.0,1.0,0.0,20.0,104.0
6.0,94.500000,272784.000000,33.0,189.0,13.000000,167.0,25.0,69.0,0.0,0.0,1.0,2.0,29.0,88.0
7.0,94.714286,308931.142857,39.0,279.0,14.000000,168.0,24.0,68.0,1.0,1.0,1.0,0.0,5.0,38.0
8.0,95.000000,271219.000000,39.0,231.0,14.000000,170.0,35.0,100.0,1.0,0.0,1.0,2.0,35.0,0.0
9.0,95.891892,245193.027027,58.0,228.0,16.000000,172.0,22.0,65.0,0.0,0.0,1.0,1.0,14.0,494.0


In [ ]:
X_duplicated = df_duplicated.drop(columns=['ID','Absenteeism time in hours'],axis=1)
y_duplicated = df_duplicated['Absenteeism time in hours']

display(X_duplicated.shape)


X_train,X_test,y_train ,y_test = train_test_split(X_duplicated,y_duplicated,random_state = 42,train_size = 0.8,test_size=0.2)
#X_train,X_test ,y_train ,y_test
lin_model = LinearRegression()
lin_model.fit(X_train,y_train)


# Make predictions using the testing set
y_pred = lin_model.predict(X_test)
y_pred_train = lin_model.predict(X_train)

print('train root Mean squared error: %.2f'
      % np.sqrt(mean_squared_error(y_train, y_pred_train)))

print('test Mean squared error: %.2f'
      % np.sqrt(mean_squared_error(y_test, y_pred)))

lin_model.score(X_test,y_test)